# Chapter 1 Examples

From "[Programming Computer Vision with Python](http://programmingcomputervision.com/)" by Jan Erik Solem (O'Reilly Media, 2012)

## 1.2 Matplotlib

"Pylab" seems to be the deprecated procedural interface to Matplotlib. The [array](https://numpy.org/doc/stable/reference/generated/numpy.array.html) function seems to be an alias to the Numpy funciton.

This [Matplotlib Image Tutorial](https://matplotlib.org/tutorials/introductory/images.html#sphx-glr-tutorials-introductory-images-py) was also helpful. **Note** that this tutorial also has an example of displaying two images side by side!

This [Medium Article](https://medium.com/dataseries/mastering-matplotlib-part-1-a480109171e3) has a good overview of the history and architecture of Matplotlib.


In [ ]:
from PIL import Image 
from pylab import *

# Increase the size of plotted images per:
# https://stackoverflow.com/questions/51937381/increase-dpi-of-matplotlib-show-in-jupyter-notebook/51937431
# rcParams['figure.dpi'] = 300
# %config InlineBackend.figure_format = 'retina'

rcParams['figure.dpi'] = 72

EMPIRE_PATH = '../example-images/empire.jpg'

# For convenience, this will always return the original image:
def empire_image():
    return Image.open(EMPIRE_PATH)

# read image to array
im = array(Image.open(EMPIRE_PATH))
im

Note that the image array above is three dimentional. The outermost array contains one array per row of pixels. The first element represents the RGB value at the upper lefthand corner of the images. I confirmed this using The Gimp and the [pointer dialog](https://docs.gimp.org/2.10/en/gimp-pointer-info-dialog.html).

In [ ]:
im[0][0] # upper lefthand corner

[imshow documentation](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imshow.html#matplotlib.pyplot.imshow)

In [ ]:
# plot the image
imshow(im)

[plot documentation](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot)

Note that the plotting functions mutate a global "sheet."

In [ ]:
# plot the image
imshow(im)

# some points
x = [100,100,400,400]
y = [200,500,200,500]

# plot the points with red star-markers
plot(x,y,'r*')

# add title and show the plot
title('Plotting: "empire.jpg"')
show()

In [ ]:

grey_im = array(Image.open(EMPIRE_PATH).convert('L'))
# create a new figure
figure()
# don’t use colors
gray() # Not sure why this is needed
# show contours with origin upper left corner
contour(grey_im, origin='image')
axis('equal')
axis('off')


Create an image histogram with 128 "bins."

In [ ]:
figure() 
hist(im.flatten(),128) 
show()

### Interactive annotation

Does not work in a Jupyter notebook.


## 1.3 NumPy

[NumPy Website](https://numpy.org/)

In [ ]:
im = array(Image.open(EMPIRE_PATH))
im.shape, im.dtype

In [ ]:
im = array(Image.open(EMPIRE_PATH).convert('L'),'f')
im.shape, im.dtype

Note that the NumPy [documentation on data types](https://numpy.org/doc/stable/user/basics.types.html) now recommends against the old one character aliases and instead the `np.` constants such as `np.float32`.

Array Slicing (page 21) has some helpful examples such as swapping rows and columns in place. It appears as though a slice with no numeric value results in an array copy. Here is an intersting example with a sum:

In [ ]:
im[:100,:50].sum()

### Graylevel transforms

In [ ]:
im = array(Image.open(EMPIRE_PATH).convert('L'))
im

In [ ]:
im2 = 255 - im #invert image
im2

In [ ]:
Image.fromarray(im2)

Numpy arrays allow for a lot of mathematical operator overloading in pretty intuative ways across multiple data types. https://www.python-course.eu/numpy_numerical_operations_on_numpy_arrays.php has a good overview.

Numpy arrays seem to be of [type ndarray](https://numpy.org/doc/stable/reference/arrays.html).

In [ ]:
isinstance(im, np.ndarray)

An interesting side note is that Numpy has 24 scalar types as described at https://numpy.org/doc/stable/reference/arrays.scalars.html

Ok, back to the book...

In [ ]:
im3 = (100.0/255) * im + 100 # clamp to interval 100...200
im3

Note that the above array can not be displayed as an image because it contains foating point numbers. Lets convert back to integers...

In [ ]:
im3_ints = im3.round(0).astype(np.uint8)
Image.fromarray(im3_ints)

In [ ]:
im4 = 255.0 * (im/255.0)**2 # squared
im4

In [ ]:
im4_ints = im4.round(0).astype(np.uint8)
Image.fromarray(im4_ints)

The same as above but emphasizing the light pixels instead of the dark ones.

In [ ]:
light_im = 255.0 * sqrt((im/255.0)) # squared
light_im

In [ ]:
Image.fromarray(light_im.round(0).astype(np.uint8))

### Image resizing

In [ ]:
# This method of importing a relative module is taken from 
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import imtools

small_light_im = imtools.imresize(light_im, (40, 40))
Image.fromarray(small_light_im)

### Histogram equalization

In [ ]:
def histeq(im,nbr_bins=256):
  """ Histogram equalization of a grayscale image. """
  
  # get image histogram
  imhist,bins = histogram(im.flatten(),nbr_bins,normed=True) 
  cdf = imhist.cumsum() # cumulative distribution function 
  cdf = 255 * cdf / cdf[-1] # normalize

  # use linear interpolation of cdf to find new pixel values
  im2 = interp(im.flatten(),bins[:-1],cdf) 
  
  return im2.reshape(im.shape), cdf

In [ ]:
im_a = array(Image.open(EMPIRE_PATH).convert('L'))

im_a_equalized, cdf = histeq(im)

im_equalized = Image.fromarray(im_a_equalized.round(0).astype(np.uint8))
im_equalized

### Multiple Subplots

Adapted from https://matplotlib.org/3.3.3/gallery/subplots_axes_and_figures/subplot.html#sphx-glr-gallery-subplots-axes-and-figures-subplot-py

In [ ]:
%matplotlib inline

rcParams['figure.dpi'] = 300

x1 = np.linspace(0.0, 5.0)
x2 = np.linspace(0.0, 2.0)

y1 = np.cos(2 * np.pi * x1) * np.exp(-x1)
y2 = np.cos(2 * np.pi * x2)

fig, (ax1, ax2) = plt.subplots(1, 2)
# fig.suptitle('A tale of 2 subplots')

ax1.set_title('Original')
ax1.imshow(im_a)
ax1.axis('off')

ax2.set_title('Equalized')
ax2.imshow(im_a_equalized)
ax2.axis('off')

plt.show()

### Averaging Images

In [ ]:
### TODO: move this to imtools.py

import sys


def compute_average(imlist):
  """ Compute the average of a list of images. """
  
  # open first image and make into array of type float
  averageim = array(Image.open(imlist[0]), 'f')

  for imname in imlist[1:]: 
    try:
      averageim += array(Image.open(imname))
    except:
      sys.stderr.write( imname + '...skipped')
  averageim /= len(imlist)

  # return average as uint8
  return array(averageim, 'uint8')



### PCA of images

PCA seems to be a trick to reduce the size of input data but still preserve primary correlations and remove the noise. There is a lot of math here. The following videos helped to refresh me on the idea of a projection matrix:

1. [Introduction to projections](https://www.youtube.com/watch?v=27vT-NWuw0M)
2. [Expressing a projection on to a line as a matrix vector prod](https://www.youtube.com/watch?v=JK-8XNIoAkI)
3. [A projection onto a subspace is a linear transformation](https://www.youtube.com/watch?v=cTyNpXB92bQ)

https://towardsdatascience.com/the-mathematics-behind-principal-component-analysis-fff2d7f4b643 also provided a good mathematical overview. Also, this [Khan Academy Video](https://www.khanacademy.org/math/ap-statistics/summarizing-quantitative-data-ap/more-standard-deviation/v/review-and-intuition-why-we-divide-by-n-1-for-the-unbiased-sample-variance) was helpful in explaining why `n - 1` is used as the divisor for covariance.

Note that the examples below require `example-images/fontimages.zip` to be extracted.


In [ ]:
from PIL import Image 
from numpy import * 
from pylab import * 
import pca

def pca_first_seven(path, ext):
    imlist = imtools.get_imlist(path, ext)
    im = array(Image.open(imlist[0])) # open one image to get size
    m,n = im.shape[0:2] # get the size of the images
    imnbr = len(imlist) # get the number of images
    # create matrix to store all flattened images
    immatrix = array([array(Image.open(im)).flatten() 
                    for im in imlist],'f')

    # perform PCA
    V,S,immean = pca.pca(immatrix)

    # show some images (mean and 7 first modes)
    figure()
    gray()
    subplot(2,4,1) 
    imshow(immean.reshape(m,n))
    for i in range(7):
      subplot(2,4,i+2)
      imshow(V[i].reshape(m,n))
      axis('off')

    show()

pca_first_seven('../example-images/a_thumbs/', 'jpg')

In [ ]:
# This exposed a bug in the PCA code where the image mean was an array of the number of pixels in the image times three.

# pca_first_seven('../example-images/annie2', 'jpg')

## 1.4 SciPy

### Bluring Images


In [ ]:
from PIL import Image
from numpy import *
from scipy.ndimage import filters

im = array(Image.open(EMPIRE_PATH).convert('L')) 
im2 = filters.gaussian_filter(im,20)
Image.fromarray(im2)


#### Color Blur

In [ ]:
im = array(Image.open(EMPIRE_PATH))
im2 = zeros(im.shape)
for i in range(3):
  # Blur blue:
  blur = 8 if i == 1 else 0 
  im2[:,:,i] = filters.gaussian_filter(im[:,:,i],blur)
im2 = uint8(im2)
Image.fromarray(im2)

### Image derivatives

In [ ]:
from PIL import Image
from numpy import *
from scipy.ndimage import filters

from lib.plot_gallery import *

im = array(Image.open(EMPIRE_PATH).convert('L'))

#Sobel derivative filters
imx = zeros(im.shape) 
filters.sobel(im,1,imx)

imy = zeros(im.shape) 
filters.sobel(im,0,imy)
magnitude = sqrt(imx**2+imy**2)

gallery = PlotGallery(columns = 4, title = 'Image Derivatives')
gallery.add_exhibit(Exhibit(im, title = 'Original'))
gallery.add_exhibit(Exhibit(abs(imx), title = 'X Derivative'))
gallery.add_exhibit(Exhibit(abs(imy), title = 'Y Derivative'))
gallery.add_exhibit(Exhibit(abs(magnitude), title = 'Magnitude'))
gallery.open()




#### Gaussian derivative filters


In [ ]:


im = array(Image.open(EMPIRE_PATH).convert('L'))
sigma = 5 #standard deviation 

imx = zeros(im.shape)
filters.gaussian_filter(
   input=im, sigma=(sigma,sigma), order=(0,1), output=imx
)

imy = zeros(im.shape)
filters.gaussian_filter(
   input=im, sigma=(sigma,sigma), order=(1,0), output=imy
)
magnitude = sqrt(imx**2+imy**2)

gallery = PlotGallery(columns = 4, title = 'Gaussian Derivative Filters (σ = ' + str(sigma) + ')')
gallery.add_exhibit(Exhibit(im, title = 'Original'))
gallery.add_exhibit(Exhibit(abs(imx), title = 'X Derivative'))
gallery.add_exhibit(Exhibit(abs(imy), title = 'Y Derivative'))
gallery.add_exhibit(Exhibit(abs(magnitude), title = 'Magnitude'))
gallery.open()



### Useful SciPy modules

#### Reading and writing .mat files

These files are binary. Note that the second argument needs to be a dict.

In [ ]:
import scipy
from scipy.io import savemat

scipy.io.savemat('/tmp/im.mat', { 'name': 'houses', 'image': im})

#### Saving arrays as images

scipy.misc.imsave seems to be deprecated.

## 1.5 Advanced example: Image de-noising



In [ ]:
from numpy import *
from numpy import random
from scipy.ndimage import filters 
import rof

SIGMA = 5

# create synthetic image with noise
im = zeros((500,500))
im[100:400,100:400] = 128
im[200:300,200:300] = 255
im = im + 30 * random.standard_normal((500,500))

U,T = rof.denoise(im,im)
G = filters.gaussian_filter(im,sigma = SIGMA)

empire = array(empire_image().convert('L'))
# empire = empire + 30 * random.standard_normal(empire.shape)
empire_gaus = filters.gaussian_filter(empire,sigma = SIGMA)
empire_denoised, T = rof.denoise(empire, empire)

gallery = PlotGallery(columns = 3, title = 'Image De-noising')
gallery.add_exhibit(Exhibit(im, title = 'Original', axis = 'on'))
gallery.add_exhibit(Exhibit(G, title = 'Gaussian Blurred (σ = ' + str(SIGMA) + ')'))
gallery.add_exhibit(Exhibit(U, title = 'ROF de-noised'))

gallery.add_exhibit(Exhibit(empire, ''))
gallery.add_exhibit(Exhibit(empire_gaus, ''))
gallery.add_exhibit(Exhibit(empire_denoised, ''))
gallery.open()
